In [1]:
from bs4 import BeautifulSoup
import asyncio
from aiohttp import ClientSession
import numpy as np
import pandas as pd
from functools import partial
from aiohttp import ClientConnectorError
import json
import re
from bs4 import NavigableString
from pprint import pprint
from datetime import datetime, timedelta
import os
from itertools import chain
import pickle

In [2]:
trails = pickle.load(open("./data/mondo_scraped/cleaned_int.pkl", "rb"))

In [3]:
trails[:10]

['http://www.elmundo.es/internacional/2017/05/26/592716a7268e3e39608b45c9.html',
 'http://www.elmundo.es/internacional/2016/09/19/57e03e3746163fe0148b460b.html',
 'http://www.elmundo.es/internacional/2015/09/05/55eadcfa46163f706b8b4578.html',
 'http://www.elmundo.es/internacional/2014/09/21/541eca59ca474105538b457a.html',
 'http://www.elmundo.es/internacional/2015/12/12/566c71c3ca47415a7e8b4677.html',
 'http://www.elmundo.es/internacional/2015/01/16/54b964d222601d1c7d8b457e.html',
 'http://www.elmundo.es/internacional/2014/05/23/537f7ffeca474196148b456d.html',
 'http://www.elmundo.es/internacional/2014/09/24/5422bea4268e3e4c618b4583.html',
 'http://www.elmundo.es/especiales/primera-guerra-mundial/vivencias/no-a-la-guerra.html',
 'http://www.elmundo.es/internacional/2016/01/30/56abc469ca4741d2358b4629.html']

In [5]:
len(trails)

20639

In [7]:
async def get_html_data(trail, session):
    """Access El Mondo webpage"""
    url = trail
    attempts = 0
    while attempts < 3:
        try:
            res = await session.get(url)
            try:
                html = await res.text()
            except UnicodeDecodeError:
                return trail, None
            try:
                soup = BeautifulSoup(html, 'html.parser')
                headline = soup.select("h1[itemprop='headline']")[0].text
                facts = [fact.text for fact in soup.select("p.summary-lead, h2[itemprop='alternativeHeadline']")]
                body = "\n".join([sent.text for sent in soup.select("div[itemprop='articleBody']")[0]
                                 .find_all('p', class_=lambda x: x != 'summary-lead')])
                tag = [tag.text for tag in soup.select("ul.item-tags a")]
            except (IndexError, TypeError) as e:
                global DEBUG
                DEBUG.append((trail, e, soup))
                print("content error!")
                return trail, None
            try:
                time = soup.select("time")[0]['datetime']
            except:
                time = None
            attrs = {
                "headline": headline,
                "keyfacts": facts, 
                "content": body,
                "tags": tag,
                "time": time
                    }
            return trail, attrs
        except ClientConnectorError:
            attempts += 1
            print("Connector error occurred!")
    if attempts == 3:
        return trail, None
    
async def gather_results(curr, step, trails):
    """Launch scrape tasks and collect results"""
    tasks = []
    async with ClientSession() as session:
        for trail in trails[curr: curr + step]:
            task = asyncio.ensure_future(get_html_data(trail, session))
            tasks.append(task)

        responses = await asyncio.gather(*tasks)
        # you now have all response bodies in this variable
        return responses


def process_df(future, curr, step):
    """Save scrape results in json files"""
    cache = {k: v for k, v in future.result()}
#     cache = future.result()
    if len(cache) == 0:
        raise RuntimeError("Empty response!")
    else:
#         json.dump(cache, open("./data/scraped/scraped_{0}_{1}.json".format(curr, curr + step), "w"))
        json.dump(cache, open("./data/mondo_scraped/contents/scraped_{0}_{1}.json".format(curr, curr + step), "w"))
        print("got it! ({0}, {1})".format(curr, curr + step))

In [12]:
# df = pd.read_csv("./data/ratings.csv")
# app_ids = df.loc[:, "app_id"].astype(str)

start = 19800
# end = len(trails)
end = 19850
step = 50
DEBUG = []

for curr in range(start, end, step):
    print("loading data from {0} to {1}".format(curr, curr + step))
    loop = asyncio.get_event_loop()
    future = asyncio.ensure_future(gather_results(curr, step, trails))
    future.add_done_callback(partial(process_df, curr=curr, step=step))
    loop.run_until_complete(future)

loading data from 19800 to 19850
got it! (19800, 19850)


## convert

In [14]:
doc_list = os.listdir("./data/mondo_scraped/contents/")
articles = dict()
for file in doc_list:
    if file[-4:] == 'json':
        with open(os.path.join("./data/mondo_scraped/contents/", file), "r") as f:
            articles.update(json.load(f))

len(articles)

20639

In [18]:
clean = {k: v for k, v in articles.items() if v is not None}

In [19]:
len(clean)

20615

In [20]:
docs = pd.DataFrame.from_dict(clean, orient='index')

In [21]:
valids = docs.loc[docs['keyfacts'].astype(np.bool), :]
valids.shape

(15022, 5)

In [22]:
valids.head()

,headline,keyfacts,content,tags,time
http://www.elmundo.es/america/2014/01/02/52c4d39a22601d6f658b457c.html,Una jueza del Tribunal Supremo suspende parcia...,[ Decidió atender a los grupos conservadores y...,"El martes, antes de presidir la fiesta de fin ...",[],2014-01-02
http://www.elmundo.es/america/2014/01/02/52c4d99622601d6d658b458a.html,'La revolución cubana sigue sin compromisos co...,[ 'Jamás hemos cedido ni cederemos ante agresi...,El presidente Raúl Castro reveló que se está i...,[],2014-01-02
http://www.elmundo.es/america/2014/01/03/52c61ede268e3e3c528b456b.html,La NSA trabaja en un ordenador cuántico capaz ...,[ La información proviene de los documentos de...,La Agencia de Seguridad Nacional (NSA) trabaja...,[],2014-01-03
http://www.elmundo.es/america/2014/01/10/52cfbb62ca47415a218b456b.html,Último adiós a la ex Miss Venezuela Mónica Spe...,[ Mónica Spear y su marido fueron asesinados e...,Esta semana Venezuela ha recibido una noticia ...,[],2014-01-10
http://www.elmundo.es/america/2014/01/14/52d4b8ba268e3eb2318b456a.html,Michoacán pone en jaque al Gobierno de Peña Nieto,[ El Gobierno envía más policías y militares y...,La situación en el Estado mexicano de Michoacá...,[],2014-01-14


In [23]:
valids.to_pickle('elmondo_es.pkl')

In [36]:
valids.iloc[7, :].apply(print);

El héroe cuya historia eclipsó al presidente
[' El sargento Cory Remsburg, herido en Afganistán, invitado de honor de Obama ', ' Presenció el discurso del estado de la Unión junto a su padre y a la primera dama ']
El protagonista más aplaudido este martes en el Capitolio no fue el presidente sino el sargento Cory Remsburg, que escuchó la ovación sujetado por su padre y junto a la primera dama, en cuya tribuna siguió el discurso como invitado de honor. 
Obama y Remsburg se han encontrado en tres ocasiones: la primera en Omaha Beach durante los actos el 65º aniversario del desembarco de Normandía y las dos siguientes mientras el sargento se recuperaba de las secuelas de un atentado que sufrió en octubre de 2009 durante su décimo despliegue en Afganistán.
El presidente se reúne a menudo en privado con miembros del Ejército. Pero ninguno le ha dejado una huella tan profunda como el sargento Remsburg. Entre otras cosas por el impacto que le produjo su segundo encuentro fortuito en un hospit